In [1]:
import pinocchio as pin
import numpy as np
import os
import meshcat

# URDF model
package_dirs = './urdf/'
urdf_file = 'tara_collisions.urdf'
urdf = package_dirs + urdf_file
robot = pin.RobotWrapper.BuildFromURDF(urdf, package_dirs)

END_EFF_FRAME_ID = robot.model.getFrameId('end_effector')

# Initialize Meshcat Visualizer
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer")
    print(err)
    sys.exit(0)

viz.loadViewerModel()

# collision data
collision_data = pin.GeometryData(robot.collision_model)

# neutral position
q = pin.neutral(robot.model)
viz.display(q)

def forward_kinematics(q):
    
    # forward kinematics 
    pin.forwardKinematics(robot.model, robot.data, q)
    
    # update frame placements and collisions
    pin.updateFramePlacements(robot.model, robot.data)
    
    # update the collision 
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, collision_data, q)
    
    # current robot configuration
    viz.displayCollisions(True)
    viz.display(q)
    
    # current end-effector position
    end_effector_pos = robot.data.oMf[END_EFF_FRAME_ID].translation
    print("End-effector position: ", end_effector_pos)

def display_collision_geometry(q):
    forward_kinematics(q)


q[0] = np.radians(0)  
display_collision_geometry(q)  



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
End-effector position:  [0.07 0.   1.08]


In [2]:
import pinocchio as pin
import numpy as np
import os
import meshcat

#URDF 
package_dirs = './urdf/'
urdf_file = 'tara_collisions.urdf'
urdf = package_dirs + urdf_file
robot = pin.RobotWrapper.BuildFromURDF(urdf, package_dirs)

END_EFF_FRAME_ID = robot.model.getFrameId('end_effector')

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer")
    print(err)
    sys.exit(0)

viz.loadViewerModel()

# possible collision 
robot.collision_model.addAllCollisionPairs()

# collision data
collision_data = pin.GeometryData(robot.collision_model)

# neutral position
q = pin.neutral(robot.model)
viz.display(q)

def forward_kinematics(q):

    pin.forwardKinematics(robot.model, robot.data, q)
    
    pin.updateFramePlacements(robot.model, robot.data)
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, collision_data, q)
    
    # current configuration
    viz.display(q)
    
    # current end-effector position
    end_effector_pos = robot.data.oMf[END_EFF_FRAME_ID].translation
    print("End-effector position: ", end_effector_pos)

def check_for_collisions(q):

    # update collision 
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, collision_data, q)
    
    # check for collisions
    for i, collision_pair in enumerate(robot.collision_model.collisionPairs):
        result = pin.computeCollision(robot.collision_model, collision_data, i)
        if result: 
            print(f"Collision detected between: {collision_pair.first} and {collision_pair.second}")
            return True
    return False

def move_robot(q):
  
    if check_for_collisions(q):
        print("Cannot move, collision detected!")
    else:
        print("No collisions detected, moving robot.")
        forward_kinematics(q)

q[0] = np.radians(0)
q[1] = np.radians(0)
q[2] = np.radians(0)
q[3] = np.radians(0)  
move_robot(q)  


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
No collisions detected, moving robot.
End-effector position:  [0.07 0.   1.08]


In [3]:
import pinocchio as pin
import numpy as np
import os
import meshcat

# URDF model
package_dirs = './urdf/'
urdf_file = 'tara_collisions.urdf'
urdf = package_dirs + urdf_file
robot = pin.RobotWrapper.BuildFromURDF(urdf, package_dirs)

END_EFF_FRAME_ID = robot.model.getFrameId('end_effector')
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer")
    print(err)
    sys.exit(0)

viz.loadViewerModel()

# possible collision pairs
robot.collision_model.addAllCollisionPairs()

# collision data
collision_data = pin.GeometryData(robot.collision_model)

# neutral position
q = pin.neutral(robot.model)
viz.display(q)

def inverse_kinematics(robot, q_init, desired_pos, max_iters=100, tol=1e-4, alpha=0.1):

    q = q_init.copy()  
    
    for i in range(max_iters):
        pin.forwardKinematics(robot.model, robot.data, q)
        pin.updateFramePlacements(robot.model, robot.data)
        
        current_pos = robot.data.oMf[END_EFF_FRAME_ID].translation
        error = desired_pos - current_pos
        
        if np.linalg.norm(error) < tol:
            print(f"Converged in {i+1} iterations.")
            return q
        
        J = pin.computeFrameJacobian(robot.model, robot.data, q, END_EFF_FRAME_ID, pin.LOCAL_WORLD_ALIGNED)[:3, :]
        
        q += alpha * np.linalg.pinv(J).dot(error)
    
    print(f"Reached max iterations. Final error: {np.linalg.norm(error)}")
    return q

def check_for_collisions(q):

    # update collision 
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, collision_data, q)
    
    # check for collisions
    for i, collision_pair in enumerate(robot.collision_model.collisionPairs):
        result = pin.computeCollision(robot.collision_model, collision_data, i)
        if result:  
            print(f"Collision detected between: {collision_pair.first} and {collision_pair.second}")
            return True
    return False

def move_robot(q):

    if check_for_collisions(q):
        print("Cannot move, collision detected!")
    else:
        print("No collisions detected, moving robot.")
        forward_kinematics(q)

def forward_kinematics(q):

    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, collision_data, q)
    viz.display(q)
    
    # Get the current end-effector position
    end_effector_pos = robot.data.oMf[END_EFF_FRAME_ID].translation
    print("End-effector position: ", end_effector_pos)

def move_robot_with_ik(desired_pos):
 
    q_init = pin.neutral(robot.model)  # Start from the neutral configuration
    q_solution = inverse_kinematics(robot, q_init, desired_pos)
    
    # Check for collisions in the IK solution
    if check_for_collisions(q_solution):
        print("Cannot move, collision detected!")
    else:
        print("No collisions detected, moving robot.")
        forward_kinematics(q_solution)

desired_position = np.array([1, 5, 6])  
move_robot_with_ik(desired_position)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Reached max iterations. Final error: 5.752670911552338
No collisions detected, moving robot.
End-effector position:  [0.24215676 1.21078381 2.00643113]
